# Pocket text classification

This project aims to, somehow, automatically create tags for Pocket articles. 
It considers your currently tagged Pocket articles to determine new tags.

## Getting content from Pocket

I've already done it, so I won't cover it right now... however, I'll add it here when possible.

## Preparing our data

We have two (real) sets of data: tagged (manually) and untagged articles. These articles are of many kinds, which I usually read (or queue for eternity) on the commute to work. 

I used to tag them, but it became really hard to catch up, so I decided to do it automatically somehow.

We want to find out the correlation between the article's text (which we acquired in [Getting content from Pocket](#Getting-content-from-Pocket) and the tags I added to them. We want to find features, or stuff that seems to be the relation between article and tag (words that define the content of the article, in a way).

### Loading and playing with our data set

Opening the json file (```'item_id', 'resolved_url', 'tags'``` are from the Pocket API, `'text'` comes from running BeautifulSoup on all articles, mentioned on [Getting content from Pocket](#Getting-content-from-Pocket))

In [ ]:
import json
fd = open("tagged_text.json")
tagged_text_list = json.loads(fd.read())
print(tagged_text_list[0].keys())

#### Tokenizing our text

In [ ]:
from nltk import download, word_tokenize
download('punkt')

test_text = tagged_text_list[0].get('text')
tokens = word_tokenize(test_text)

#### Getting the most common words from our text

In [ ]:
from nltk.probability import FreqDist
fdist = FreqDist(word.lower() for word in tokens)
fdist.most_common(100)

#### Getting the least common words from our text

In [ ]:
fdist.most_common(100)[-20:]

#### Plotting some stuff

In [ ]:
# This is used so jupyter can plot
%matplotlib inline
fdist.plot()

### First possibility: get outstanding words in relation to all articles

One possibility is to try to separate common words from outstanding words.

We can tokenize all the articles and put them in the same pool of words. Then, we count how many times a word is repeated in all pool of tokens.

Common words, such as connectors, nouns, verbs, will probably be very common. In other hand, specific words will repeat less in the pool of tokens (they may repeat a lot in a single article).

In other words, we want to find words in an article that have small probability to be found in other articles.

First of all, I'll grab all the tokens from a text. Next, I'll grab only the tokens and add them to an array of token occurences (i.e. how many articles does this token appear). It will hold content such as:

```json
# token, occurences
{"word": 2}
```

In [ ]:
from nltk.probability import FreqDist
token_occurences = {}
token_probabilities = {}
# We just want to text the 3 first articles right now
# tagged_text_list = [text for text in tagged_text_list[0:3]]
# Getting all tokens occurences across texts
for article in tagged_text_list:
    content = article.get('text')
    tokens = word_tokenize(content)
    fdist = FreqDist(word.lower() for word in tokens)
    for word in fdist.most_common(100):
        if word[0] in token_occurences.keys():
            token_occurences[word[0]] = token_occurences[word[0]] + 1
        else:
            token_occurences[word[0]] = 1

# Calculating token probabilities

for token, count in token_occurences.items():
    token_probabilities[token] = count / len(tagged_text_list) * 100

# Getting words with less than 50% of probability

filtered_dict = {}
threshold = 50
for key, value in token_probabilities.items():
    if value <= threshold:
        filtered_dict[key] = value

print(filtered_dict)

Now, we want to see which most common words in an article show as less common words in our word pool.

In [ ]:
article_features = []
article = tagged_text_list[0]
content = article.get('text')
tokens = word_tokenize(content)
fdist = FreqDist(word.lower() for word in tokens)
for word in fdist.most_common(100):
    if word[0] in filtered_dict.keys():
        if word[0] not in article_features:
            article_features.append(word[0])
print(article_features)